# Step 1: Extract Imagery Data for Lake Michigan from GOES

This is the 1st step of the experiment. Before get started, you should obtain the visible band data generated by GOES Satellite service. The original file is in `.nc` format, and it contains the full scan of the continent. However, in our application, it is not needed. Therefore, we choose to use NOAA's Weather and Climate Toolkit (Viewer and Data Exporter) to extract the data for the area which we are interested in. The files exported by the toolkit is in `.csv` format.

In [1]:
!pip install scipy # Just in case.

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import pandas as pd
import scipy
import numpy as np

In [3]:
from scipy.stats import skew


### TO-DO:

In this part, please change the directory path to the `.csv` files generated by NOAA's toolkit.

In [4]:
os.getcwd()
os.chdir('GOES_Hourly_Statistics/raw_csv/2015Fall_2016Spring')
os.getcwd()

'/srv/scratch/NOAA/GOES_Hourly_Statistics/raw_csv/2015Fall_2016Spring'

In [5]:
len(os.listdir())

15687

In [6]:
filename = os.listdir()
len(filename)

15687

In [7]:
filename[0:10]

['goes15.2016.02.13.1330.v01.nc-var1-t0.csv',
 'goes15.2015.12.24.1900.v01.nc-var1-t0.csv',
 'goes15.2016.02.10.0100.v01.nc-var1-t0.csv',
 'goes15.2016.01.30.0330.v01.nc-var1-t0.csv',
 'goes15.2016.02.06.1030.v01.nc-var1-t0.csv',
 'goes15.2015.11.27.0945.v01.nc-var1-t0.csv',
 'goes15.2015.12.15.2200.v01.nc-var1-t0.csv',
 'goes15.2016.01.07.2330.v01.nc-var1-t0.csv',
 'goes15.2016.01.23.1200.v01.nc-var1-t0.csv',
 'goes15.2016.01.01.2300.v01.nc-var1-t0.csv']

**Important: Sort is a must for linux system.**

In [8]:
filename.sort()

In [9]:
## Quick inspection of the order.

filename[0:10]

['goes15.2015.10.01.0000.v01.nc-var1-t0.csv',
 'goes15.2015.10.01.0030.v01.nc-var1-t0.csv',
 'goes15.2015.10.01.0100.v01.nc-var1-t0.csv',
 'goes15.2015.10.01.0115.v01.nc-var1-t0.csv',
 'goes15.2015.10.01.0130.v01.nc-var1-t0.csv',
 'goes15.2015.10.01.0145.v01.nc-var1-t0.csv',
 'goes15.2015.10.01.0200.v01.nc-var1-t0.csv',
 'goes15.2015.10.01.0215.v01.nc-var1-t0.csv',
 'goes15.2015.10.01.0230.v01.nc-var1-t0.csv',
 'goes15.2015.10.01.0245.v01.nc-var1-t0.csv']

### TO-DO:

The `Lake_Partition.csv` contains a table of the boundary values which can be used to precisely extract the data from Lake Michigan area.

<img src="Zone_partition.png" alt="partition" width="350"/>

In [10]:
## TO-DO: Change the directory if needed.
LP = pd.read_csv('/srv/scratch/NOAA/Lake_Partition.csv')
LP

,latitude,first_bound,second_bound,left_zone,mid_zone,right_zone
0,40.06,-87.5465,-84.8170,2,3,4
1,40.10,-87.5465,-84.8170,2,3,4
2,40.14,-87.5465,-84.8170,2,3,4
3,40.18,-87.5465,-84.8170,2,3,4
4,40.22,-87.5465,-84.8170,2,3,4
...,...,...,...,...,...,...
193,47.78,-86.1781,-84.9473,6,6,6
194,47.82,-86.1781,-84.9473,6,6,6
195,47.86,-86.1781,-84.9473,6,6,6
196,47.90,-86.1781,-84.9473,6,6,6


In [11]:
# Optional: Check the list of filenames one more time.
for i in filename[:3]:
    print(i)

goes15.2015.10.01.0000.v01.nc-var1-t0.csv
goes15.2015.10.01.0030.v01.nc-var1-t0.csv
goes15.2015.10.01.0100.v01.nc-var1-t0.csv


### TO-DO:

Make sure to change the directory of the outputs!

In [12]:
!pwd

/srv/scratch/NOAA/GOES_Hourly_Statistics/raw_csv/2015Fall_2016Spring


In [13]:
#!mkdir /srv/scratch/NOAA/GOES_Hourly_Statistics/zone_0_2013Fall_2014Spring/

In [14]:
for fn in filename[:3000]:
    file = pd.read_csv(fn)
    flp = pd.merge(file, LP, on = 'latitude')
    par_list = []
    for i in range(len(flp)):
        a = flp['longitude'][i] 
        b = flp['first_bound'][i]
        c = flp['second_bound'][i]
        if a < b:
            par_list.append(flp['left_zone'][i])
        elif a > c:
            par_list.append(flp['right_zone'][i])
        else:
            par_list.append(flp['mid_zone'][i])
    flp['partition'] = par_list
    flp = flp.loc[:, ['value', 'datetime', 'latitude', 'longitude', 'partition']]
    s_flp = flp[flp['partition'] == 0].reset_index().rename(columns={'index': 'corresponding row'})
    
    ## TO-DO: Change the directory, clearly lable as "zone_0"
    s_flp.to_csv('/srv/scratch/NOAA/GOES_Hourly_Statistics/zone_0_2015Fall_2016Spring/'+str(fn),index = False)
    print(fn)



goes15.2015.10.01.0000.v01.nc-var1-t0.csv
goes15.2015.10.01.0030.v01.nc-var1-t0.csv
goes15.2015.10.01.0100.v01.nc-var1-t0.csv
goes15.2015.10.01.0115.v01.nc-var1-t0.csv
goes15.2015.10.01.0130.v01.nc-var1-t0.csv
goes15.2015.10.01.0145.v01.nc-var1-t0.csv
goes15.2015.10.01.0200.v01.nc-var1-t0.csv
goes15.2015.10.01.0215.v01.nc-var1-t0.csv
goes15.2015.10.01.0230.v01.nc-var1-t0.csv
goes15.2015.10.01.0245.v01.nc-var1-t0.csv
goes15.2015.10.01.0300.v01.nc-var1-t0.csv
goes15.2015.10.01.0330.v01.nc-var1-t0.csv
goes15.2015.10.01.0345.v01.nc-var1-t0.csv
goes15.2015.10.01.0400.v01.nc-var1-t0.csv
goes15.2015.10.01.0415.v01.nc-var1-t0.csv
goes15.2015.10.01.0430.v01.nc-var1-t0.csv
goes15.2015.10.01.0445.v01.nc-var1-t0.csv
goes15.2015.10.01.0500.v01.nc-var1-t0.csv
goes15.2015.10.01.0515.v01.nc-var1-t0.csv
goes15.2015.10.01.0530.v01.nc-var1-t0.csv
goes15.2015.10.01.0545.v01.nc-var1-t0.csv
goes15.2015.10.01.0600.v01.nc-var1-t0.csv
goes15.2015.10.01.0630.v01.nc-var1-t0.csv
goes15.2015.10.01.0645.v01.nc-var1

goes15.2015.10.03.0715.v01.nc-var1-t0.csv
goes15.2015.10.03.0730.v01.nc-var1-t0.csv
goes15.2015.10.03.0745.v01.nc-var1-t0.csv
goes15.2015.10.03.0800.v01.nc-var1-t0.csv
goes15.2015.10.03.0815.v01.nc-var1-t0.csv
goes15.2015.10.03.0830.v01.nc-var1-t0.csv
goes15.2015.10.03.0845.v01.nc-var1-t0.csv
goes15.2015.10.03.0900.v01.nc-var1-t0.csv
goes15.2015.10.03.0930.v01.nc-var1-t0.csv
goes15.2015.10.03.0945.v01.nc-var1-t0.csv
goes15.2015.10.03.1000.v01.nc-var1-t0.csv
goes15.2015.10.03.1015.v01.nc-var1-t0.csv
goes15.2015.10.03.1030.v01.nc-var1-t0.csv
goes15.2015.10.03.1045.v01.nc-var1-t0.csv
goes15.2015.10.03.1100.v01.nc-var1-t0.csv
goes15.2015.10.03.1115.v01.nc-var1-t0.csv
goes15.2015.10.03.1130.v01.nc-var1-t0.csv
goes15.2015.10.03.1145.v01.nc-var1-t0.csv
goes15.2015.10.03.1200.v01.nc-var1-t0.csv
goes15.2015.10.03.1230.v01.nc-var1-t0.csv
goes15.2015.10.03.1245.v01.nc-var1-t0.csv
goes15.2015.10.03.1300.v01.nc-var1-t0.csv
goes15.2015.10.03.1315.v01.nc-var1-t0.csv
goes15.2015.10.03.1330.v01.nc-var1

goes15.2015.10.05.1415.v01.nc-var1-t0.csv
goes15.2015.10.05.1430.v01.nc-var1-t0.csv
goes15.2015.10.05.1445.v01.nc-var1-t0.csv
goes15.2015.10.05.1500.v01.nc-var1-t0.csv
goes15.2015.10.05.1530.v01.nc-var1-t0.csv
goes15.2015.10.05.1545.v01.nc-var1-t0.csv
goes15.2015.10.05.1600.v01.nc-var1-t0.csv
goes15.2015.10.05.1615.v01.nc-var1-t0.csv
goes15.2015.10.05.1630.v01.nc-var1-t0.csv
goes15.2015.10.05.1645.v01.nc-var1-t0.csv
goes15.2015.10.05.1700.v01.nc-var1-t0.csv
goes15.2015.10.05.1715.v01.nc-var1-t0.csv
goes15.2015.10.05.1730.v01.nc-var1-t0.csv
goes15.2015.10.05.1745.v01.nc-var1-t0.csv
goes15.2015.10.05.1800.v01.nc-var1-t0.csv
goes15.2015.10.05.1830.v01.nc-var1-t0.csv
goes15.2015.10.05.1845.v01.nc-var1-t0.csv
goes15.2015.10.05.1900.v01.nc-var1-t0.csv
goes15.2015.10.05.1915.v01.nc-var1-t0.csv
goes15.2015.10.05.1930.v01.nc-var1-t0.csv
goes15.2015.10.05.1945.v01.nc-var1-t0.csv
goes15.2015.10.05.2000.v01.nc-var1-t0.csv
goes15.2015.10.05.2015.v01.nc-var1-t0.csv
goes15.2015.10.05.2030.v01.nc-var1

goes15.2015.10.07.2145.v01.nc-var1-t0.csv
goes15.2015.10.07.2200.v01.nc-var1-t0.csv
goes15.2015.10.07.2215.v01.nc-var1-t0.csv
goes15.2015.10.07.2230.v01.nc-var1-t0.csv
goes15.2015.10.07.2245.v01.nc-var1-t0.csv
goes15.2015.10.07.2300.v01.nc-var1-t0.csv
goes15.2015.10.07.2315.v01.nc-var1-t0.csv
goes15.2015.10.07.2330.v01.nc-var1-t0.csv
goes15.2015.10.07.2345.v01.nc-var1-t0.csv
goes15.2015.10.08.0000.v01.nc-var1-t0.csv
goes15.2015.10.08.0030.v01.nc-var1-t0.csv
goes15.2015.10.08.0100.v01.nc-var1-t0.csv
goes15.2015.10.08.0115.v01.nc-var1-t0.csv
goes15.2015.10.08.0130.v01.nc-var1-t0.csv
goes15.2015.10.08.0145.v01.nc-var1-t0.csv
goes15.2015.10.08.0200.v01.nc-var1-t0.csv
goes15.2015.10.08.0215.v01.nc-var1-t0.csv
goes15.2015.10.08.0230.v01.nc-var1-t0.csv
goes15.2015.10.08.0245.v01.nc-var1-t0.csv
goes15.2015.10.08.0300.v01.nc-var1-t0.csv
goes15.2015.10.08.0330.v01.nc-var1-t0.csv
goes15.2015.10.08.0345.v01.nc-var1-t0.csv
goes15.2015.10.08.0400.v01.nc-var1-t0.csv
goes15.2015.10.08.0415.v01.nc-var1

goes15.2015.10.10.0800.v01.nc-var1-t0.csv
goes15.2015.10.10.0815.v01.nc-var1-t0.csv
goes15.2015.10.10.0830.v01.nc-var1-t0.csv
goes15.2015.10.10.0845.v01.nc-var1-t0.csv
goes15.2015.10.10.0900.v01.nc-var1-t0.csv
goes15.2015.10.10.0930.v01.nc-var1-t0.csv
goes15.2015.10.10.0945.v01.nc-var1-t0.csv
goes15.2015.10.10.1000.v01.nc-var1-t0.csv
goes15.2015.10.10.1015.v01.nc-var1-t0.csv
goes15.2015.10.10.1030.v01.nc-var1-t0.csv
goes15.2015.10.10.1045.v01.nc-var1-t0.csv
goes15.2015.10.10.1100.v01.nc-var1-t0.csv
goes15.2015.10.10.1115.v01.nc-var1-t0.csv
goes15.2015.10.10.1130.v01.nc-var1-t0.csv
goes15.2015.10.10.1145.v01.nc-var1-t0.csv
goes15.2015.10.10.1200.v01.nc-var1-t0.csv
goes15.2015.10.10.1230.v01.nc-var1-t0.csv
goes15.2015.10.10.1245.v01.nc-var1-t0.csv
goes15.2015.10.10.1300.v01.nc-var1-t0.csv
goes15.2015.10.10.1315.v01.nc-var1-t0.csv
goes15.2015.10.10.1330.v01.nc-var1-t0.csv
goes15.2015.10.10.1345.v01.nc-var1-t0.csv
goes15.2015.10.10.1400.v01.nc-var1-t0.csv
goes15.2015.10.10.1415.v01.nc-var1

...

goes15.2015.10.26.0700.v01.nc-var1-t0.csv
goes15.2015.10.26.0715.v01.nc-var1-t0.csv
goes15.2015.10.26.0730.v01.nc-var1-t0.csv
goes15.2015.10.26.0745.v01.nc-var1-t0.csv
goes15.2015.10.26.0800.v01.nc-var1-t0.csv
goes15.2015.10.26.0815.v01.nc-var1-t0.csv
goes15.2015.10.26.0830.v01.nc-var1-t0.csv
goes15.2015.10.26.0845.v01.nc-var1-t0.csv
goes15.2015.10.26.0900.v01.nc-var1-t0.csv
goes15.2015.10.26.0930.v01.nc-var1-t0.csv
goes15.2015.10.26.0945.v01.nc-var1-t0.csv
goes15.2015.10.26.1000.v01.nc-var1-t0.csv
goes15.2015.10.26.1015.v01.nc-var1-t0.csv
goes15.2015.10.26.1030.v01.nc-var1-t0.csv
goes15.2015.10.26.1045.v01.nc-var1-t0.csv
goes15.2015.10.26.1100.v01.nc-var1-t0.csv
goes15.2015.10.26.1115.v01.nc-var1-t0.csv
goes15.2015.10.26.1130.v01.nc-var1-t0.csv
goes15.2015.10.26.1145.v01.nc-var1-t0.csv
goes15.2015.10.26.1200.v01.nc-var1-t0.csv
goes15.2015.10.26.1230.v01.nc-var1-t0.csv
goes15.2015.10.26.1245.v01.nc-var1-t0.csv
goes15.2015.10.26.1300.v01.nc-var1-t0.csv
goes15.2015.10.26.1315.v01.nc-var1

goes15.2015.10.28.1330.v01.nc-var1-t0.csv
goes15.2015.10.28.1345.v01.nc-var1-t0.csv
goes15.2015.10.28.1400.v01.nc-var1-t0.csv
goes15.2015.10.28.1415.v01.nc-var1-t0.csv
goes15.2015.10.28.1430.v01.nc-var1-t0.csv
goes15.2015.10.28.1445.v01.nc-var1-t0.csv
goes15.2015.10.28.1500.v01.nc-var1-t0.csv
goes15.2015.10.28.1530.v01.nc-var1-t0.csv
goes15.2015.10.28.1545.v01.nc-var1-t0.csv
goes15.2015.10.28.1600.v01.nc-var1-t0.csv
goes15.2015.10.28.1615.v01.nc-var1-t0.csv
goes15.2015.10.28.1630.v01.nc-var1-t0.csv
goes15.2015.10.28.1645.v01.nc-var1-t0.csv
goes15.2015.10.28.1700.v01.nc-var1-t0.csv
goes15.2015.10.28.1715.v01.nc-var1-t0.csv
goes15.2015.10.28.1730.v01.nc-var1-t0.csv
goes15.2015.10.28.1745.v01.nc-var1-t0.csv
goes15.2015.10.28.1800.v01.nc-var1-t0.csv
goes15.2015.10.28.1830.v01.nc-var1-t0.csv
goes15.2015.10.28.1845.v01.nc-var1-t0.csv
goes15.2015.10.28.1900.v01.nc-var1-t0.csv
goes15.2015.10.28.1915.v01.nc-var1-t0.csv
goes15.2015.10.28.1930.v01.nc-var1-t0.csv
goes15.2015.10.28.1945.v01.nc-var1

goes15.2015.10.30.2015.v01.nc-var1-t0.csv
goes15.2015.10.30.2030.v01.nc-var1-t0.csv
goes15.2015.10.30.2100.v01.nc-var1-t0.csv
goes15.2015.10.30.2130.v01.nc-var1-t0.csv
goes15.2015.10.30.2145.v01.nc-var1-t0.csv
goes15.2015.10.30.2200.v01.nc-var1-t0.csv
goes15.2015.10.30.2215.v01.nc-var1-t0.csv
goes15.2015.10.30.2230.v01.nc-var1-t0.csv
goes15.2015.10.30.2245.v01.nc-var1-t0.csv
goes15.2015.10.30.2300.v01.nc-var1-t0.csv
goes15.2015.10.30.2315.v01.nc-var1-t0.csv
goes15.2015.10.30.2330.v01.nc-var1-t0.csv
goes15.2015.10.30.2345.v01.nc-var1-t0.csv
goes15.2015.10.31.0000.v01.nc-var1-t0.csv
goes15.2015.10.31.0030.v01.nc-var1-t0.csv
goes15.2015.10.31.0100.v01.nc-var1-t0.csv
goes15.2015.10.31.0115.v01.nc-var1-t0.csv
goes15.2015.10.31.0130.v01.nc-var1-t0.csv
goes15.2015.10.31.0145.v01.nc-var1-t0.csv
goes15.2015.10.31.0200.v01.nc-var1-t0.csv
goes15.2015.10.31.0215.v01.nc-var1-t0.csv
goes15.2015.10.31.0230.v01.nc-var1-t0.csv
goes15.2015.10.31.0245.v01.nc-var1-t0.csv
goes15.2015.10.31.0300.v01.nc-var1

goes15.2015.11.02.0330.v01.nc-var1-t0.csv
goes15.2015.11.02.0345.v01.nc-var1-t0.csv
goes15.2015.11.02.0400.v01.nc-var1-t0.csv
goes15.2015.11.02.0415.v01.nc-var1-t0.csv
goes15.2015.11.02.0430.v01.nc-var1-t0.csv
goes15.2015.11.02.0445.v01.nc-var1-t0.csv
goes15.2015.11.02.0500.v01.nc-var1-t0.csv
goes15.2015.11.02.0515.v01.nc-var1-t0.csv
goes15.2015.11.02.0530.v01.nc-var1-t0.csv
goes15.2015.11.02.0545.v01.nc-var1-t0.csv
goes15.2015.11.02.0600.v01.nc-var1-t0.csv
goes15.2015.11.02.0630.v01.nc-var1-t0.csv
goes15.2015.11.02.0645.v01.nc-var1-t0.csv
goes15.2015.11.02.0700.v01.nc-var1-t0.csv
goes15.2015.11.02.0715.v01.nc-var1-t0.csv
goes15.2015.11.02.0730.v01.nc-var1-t0.csv
goes15.2015.11.02.0745.v01.nc-var1-t0.csv
goes15.2015.11.02.0800.v01.nc-var1-t0.csv
goes15.2015.11.02.0815.v01.nc-var1-t0.csv
goes15.2015.11.02.0830.v01.nc-var1-t0.csv
goes15.2015.11.02.0845.v01.nc-var1-t0.csv
goes15.2015.11.02.0900.v01.nc-var1-t0.csv
goes15.2015.11.02.0930.v01.nc-var1-t0.csv
goes15.2015.11.02.0945.v01.nc-var1

goes15.2015.11.04.0945.v01.nc-var1-t0.csv
goes15.2015.11.04.1000.v01.nc-var1-t0.csv
goes15.2015.11.04.1015.v01.nc-var1-t0.csv
goes15.2015.11.04.1030.v01.nc-var1-t0.csv
goes15.2015.11.04.1045.v01.nc-var1-t0.csv
goes15.2015.11.04.1100.v01.nc-var1-t0.csv
goes15.2015.11.04.1115.v01.nc-var1-t0.csv
goes15.2015.11.04.1130.v01.nc-var1-t0.csv
goes15.2015.11.04.1145.v01.nc-var1-t0.csv
goes15.2015.11.04.1200.v01.nc-var1-t0.csv
goes15.2015.11.04.1230.v01.nc-var1-t0.csv
goes15.2015.11.04.1245.v01.nc-var1-t0.csv
goes15.2015.11.04.1300.v01.nc-var1-t0.csv
goes15.2015.11.04.1315.v01.nc-var1-t0.csv
goes15.2015.11.04.1330.v01.nc-var1-t0.csv
goes15.2015.11.04.1345.v01.nc-var1-t0.csv
goes15.2015.11.04.1400.v01.nc-var1-t0.csv
goes15.2015.11.04.1415.v01.nc-var1-t0.csv
goes15.2015.11.04.1430.v01.nc-var1-t0.csv
goes15.2015.11.04.1445.v01.nc-var1-t0.csv
goes15.2015.11.04.1500.v01.nc-var1-t0.csv
goes15.2015.11.04.1530.v01.nc-var1-t0.csv
goes15.2015.11.04.1545.v01.nc-var1-t0.csv
goes15.2015.11.04.1600.v01.nc-var1